In [0]:
USE data_science.default

用最近90天的cost，最近120-180天前的收入，对比原始表和纯利表，推测回本目标

In [0]:
CREATE OR REPLACE view lw_20250903_kpi_target_view_by_j as
WITH profit AS (
    SELECT
        app_package,
        country_group,
        mediasource,
        install_day,
        cost,
        revenue_d120,
        revenue_d135
    FROM lw_20250903_aos_gpir_cohort_onlyprofit_raw_table_by_j
    UNION ALL
    SELECT
        app_package,
        country_group,
        mediasource,
        install_day,
        cost,
        revenue_d120,
        revenue_d135
    FROM lw_20250903_ios_af_cohort_onlyprofit_raw_view_by_j
),
raw AS (
    SELECT
        app_package,
        country_group,
        mediasource,
        install_day,
        cost,
        revenue_d120,
        revenue_d135
    FROM lw_20250903_aos_gpir_cohort_raw_table_by_j
    UNION ALL
    SELECT
        app_package,
        country_group,
        mediasource,
        install_day,
        cost,
        revenue_d120,
        revenue_d135
    FROM lw_20250903_ios_af_cohort_raw_view_by_j
),
-- 1. 按原分组获取最近90天的花费汇总
cost_agg_data AS (
    SELECT
        app_package,
        country_group,
        mediasource,
        install_day,
        -- 原始数据最近90天成本汇总
        SUM(cost) OVER (
            PARTITION BY app_package, country_group, mediasource
            ORDER BY to_date(install_day, 'yyyyMMdd')
            ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
        ) AS last90_cost_sum
    FROM raw
),
profit_cost_agg AS (
    SELECT
        app_package,
        country_group,
        mediasource,
        install_day,
        -- profit数据最近90天成本汇总
        SUM(cost) OVER (
            PARTITION BY app_package, country_group, mediasource
            ORDER BY to_date(install_day, 'yyyyMMdd')
            ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
        ) AS last90_cost_p_sum
    FROM profit
),
-- 2. 只按app_package、country_group分组获取120-180天前的收入汇总
revenue_agg_data AS (
    SELECT
        app_package,
        country_group,
        install_day,
        -- 计算120-180天前的收入汇总
        SUM(SUM(revenue_d120)) OVER (
            PARTITION BY app_package, country_group
            ORDER BY to_date(install_day, 'yyyyMMdd')
            ROWS BETWEEN 179 PRECEDING AND 120 PRECEDING
        ) AS revenue_120to180_sum
    FROM raw
    GROUP BY app_package, country_group, install_day
),
profit_revenue_agg AS (
    SELECT
        app_package,
        country_group,
        install_day,
        -- 计算120-180天前的profit收入汇总
        SUM(SUM(revenue_d120)) OVER (
            PARTITION BY app_package, country_group
            ORDER BY to_date(install_day, 'yyyyMMdd')
            ROWS BETWEEN 179 PRECEDING AND 120 PRECEDING
        ) AS revenue_120to180_p_sum
    FROM profit
    GROUP BY app_package, country_group, install_day
),
-- 3. 将花费和收入数据join在一起
final_joined AS (
    SELECT
        c.app_package,
        c.country_group,
        c.mediasource,
        c.install_day,
        c.last90_cost_sum,
        pc.last90_cost_p_sum,
        r.revenue_120to180_sum,
        pr.revenue_120to180_p_sum
    FROM cost_agg_data c
    LEFT JOIN profit_cost_agg pc ON 
        c.app_package = pc.app_package
        AND c.country_group = pc.country_group
        AND c.mediasource = pc.mediasource
        AND c.install_day = pc.install_day
    LEFT JOIN revenue_agg_data r ON 
        c.app_package = r.app_package
        AND c.country_group = r.country_group
        AND c.install_day = r.install_day
    LEFT JOIN profit_revenue_agg pr ON 
        c.app_package = pr.app_package
        AND c.country_group = pr.country_group
        AND c.install_day = pr.install_day
),
-- 4. 计算KPI目标值
kpi_calculated AS (
    SELECT
        app_package,
        country_group,
        mediasource,
        install_day,
        -- 计算KPI目标值
        CASE
            WHEN last90_cost_sum > 0 
            AND revenue_120to180_p_sum > 0 
            AND last90_cost_p_sum > 0
            AND revenue_120to180_sum > 0
            THEN (revenue_120to180_sum / last90_cost_sum) / (revenue_120to180_p_sum / last90_cost_p_sum)
            ELSE NULL
        END AS kpi_target
    FROM final_joined
)
SELECT
    app_package,
    country_group,
    mediasource,
    install_day,
    kpi_target
FROM kpi_calculated
WHERE kpi_target IS NOT NULL
ORDER BY app_package, country_group, mediasource, to_date(install_day, 'yyyyMMdd');

In [0]:
-- 对iOS的kpi_target进行修正
create or replace view lw_20250903_kpi_target_fit_view_by_j as
with bayesian_results AS (
	-- 获取拟合系数和自然量收入
	SELECT
		country_group,
		organic_revenue,
		applovin_int_d7_coeff,
		applovin_int_d28_coeff,
		facebook_ads_coeff,
		moloco_int_coeff,
		bytedanceglobal_int_coeff,
		tag
	FROM
		lw_20250703_ios_bayesian_result_by_j
	WHERE
		tag IN ('20250808_20')
)
select
  app_package,
  kt.country_group,
  mediasource,
  install_day,
  -- kpi_target,
  CASE
    WHEN mediasource = 'applovin_int_d7' THEN try_divide(kpi_target , COALESCE(br.applovin_int_d7_coeff, 1.0))
    WHEN mediasource = 'applovin_int_d28' THEN try_divide(kpi_target , COALESCE(br.applovin_int_d28_coeff, 1.0))
    WHEN mediasource = 'Facebook Ads' THEN try_divide(kpi_target , COALESCE(br.facebook_ads_coeff, 1.0))
    WHEN mediasource = 'moloco_int' THEN try_divide(kpi_target , COALESCE(br.moloco_int_coeff, 1.0))
    WHEN mediasource = 'bytedanceglobal_int' THEN try_divide(kpi_target , COALESCE(br.bytedanceglobal_int_coeff, 1.0))
    ELSE kpi_target
  END AS kpi_target
from lw_20250903_kpi_target_view_by_j kt
left join bayesian_results br on
kt.country_group = br.country_group
where 
app_package = 'id6448786147'

union all
select 
  app_package,
  country_group,
  mediasource,
  install_day,
  kpi_target
from lw_20250903_kpi_target_view_by_j
where
app_package = 'com.fun.lastwar.gp'
;



In [0]:
create or replace table lw_20250903_kpi_target_table_by_j
select * from lw_20250903_kpi_target_fit_view_by_j